In [21]:
###score probalisitc forecast
import pandas as pd
import sklearn
from sklearn.metrics import log_loss, brier_score_loss
import os
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#Input data in format 

locations = [ "United Kingdom","Japan"]
dates = ['2022-01-24', '2022-02-04','2022-02-08','2022-02-18','2022-02-23',
         '2022-02-28','2022-03-03','2022-03-08','2022-03-15',
         '2022-03-21','2022-03-25','2022-04-07','2022-04-14','2022-04-27'
         ,'2022-05-06','2022-05-17','2022-05-20','2022-05-28','2022-06-09'
         ,'2022-06-14','2022-06-22']

#Latest model run "truth"
truth_set = pd.read_csv("https://raw.githubusercontent.com/blab/rt-from-frequency-dynamics/master/estimates/omicron-countries-split/omicron-countries-split_freq-combined-GARW.tsv", sep="\t")
final_truth = truth_set.rename(columns = {'median_freq':'truth_freq'}, inplace = False)
final_truth = final_truth[final_truth['location']=='United Kingdom']
final_truth = final_truth[['date','location','variant','truth_freq']]


#full model output set dict


final_sets = {}
pred_dic = {}
for date in dates:
    filepath = f"cast_estimates_full/United Kingdom/freq_full_{date}.csv"
    
    #Check if file exists and continue if not
    if not os.path.exists(filepath):
        continue
    #read models and add to dict
    pred_dic[date] = pd.read_csv(f"cast_estimates_full/United Kingdom/freq_full_{date}.csv")

    #loop through data and merge to final set
final_sets = {k: pd.merge(final_truth,d) for k,d in pred_dic.items()} 
#print(final_sets.keys())

print(final_sets)          
                            



In [ ]:
def prep_freq_data(final_set):
    
    
    #return truth values as np.arrays
    
    truth_values = final_set['truth_freq'].to_numpy()
    
    #return nowcast and forecast values as np.arrays
    
    nowcast_values = final_set['median_freq_nowcast'].to_numpy()
    
    forecast_values = final_set['median_freq_forecast'].to_numpy()
    
    return truth_values, nowcast_values, forecast_values


#prep arrays of data
prepped_data = {k: prep_freq_data(v) for k,v in final_sets.items()}
print(prepped_data)


In [5]:
#Class scores to get scores
from abc import ABC, abstractmethod

In [ ]:
class Scores(ABC):
    @abstractmethod
    def __init__(self):
        pass
class MAE(Scores):
    def __init__(self):
        pass
    def evaluate(self,truth, prediction):
        abs_error = np.abs(truth - prediction)
        return np.nanmean(abs_error)
        
class MSE(Scores):
    def __init__(self):
        pass
    def evaluate(self, truth, prediction):
        squared_error = np.square(truth - prediction)
        return np.nanmean(squared_error)
        
class LogLoss(Scores):
    def __init__(self):
        pass
        #mlr log loss error
    def evaluate(self, truth, prediction):
        log_loss = sklearn.metrics.log_loss(self.truth, prediction, eps=1e-15, normalize=True, sample_weight=None, labels=None)
        return log_loss


    
error_id_dict = {}    
for k, v in prepped_data.items():
    error_dict={}
    #MAE
    
    error = MAE()  
    error_dict['MAE'] = error.evaluate(v[0],v[1])
    
    #MSE
    mse = MSE()
    error_dict['MSE'] = mse.evaluate(v[0],v[1])
    
    error_id_dict[k] = error_dict
    
print(error_id_dict)

In [ ]:
#Summarize results

out = []

for k,v in error_id_dict.items():
    #print(v)
    v['MAE']
    #v['MSE']
    out.append(v['MAE'])
    #out.append(v['MSE'])

print(out)